In [ ]:
# Imports
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

plt.rcParams['figure.figsize'] = (15.0, 8.0) # set default size of plots
plt.rcParams['figure.facecolor'] = 'white'

pd.set_option('display.max_rows', None)

matplotlib.rcParams.update({'font.size': 15})

In [ ]:
TIME_RETRIEVAL_REE = 0
TIME_RETRIEVAL_TEE = 10
TIME_RETRIEVAL_REE_WASM = 0
TIME_RETRIEVAL_TEE_WASM = 13

In [ ]:
def compute_mean_std(size, time, factor = 1):
    df = pd.DataFrame(columns = ["Size","Time"])
    df["Size"] = size
    df["Time"] = time * factor
    
    result = pd.DataFrame(columns = ["Mean","Std"])
    result["Mean"] = df.groupby("Size")["Time"].median() #df.groupby("Size")["Time"].apply(stats.gmean)
    result["Std"] = df.groupby("Size")["Time"].std() #df.groupby("Size")["Time"].apply(stats.gstd)
    #result["Size"] = df.groupby("Size").groups.keys()
    
    return result

def formatForLatex(n, digits_round = 2):
    roundedAsString = str(round(n, digits_round))
    if "." in roundedAsString:
        roundedAsString = roundedAsString.rstrip("0")
        roundedAsString = roundedAsString.rstrip(".")
    return f"{roundedAsString}"

In [ ]:
#
## Attester data
#

In [ ]:
# Import the dataset
attester = pd.read_csv("../logs/message3-time/attester.csv", \
        names = ["Size","MallocStart","MallocEnd","DecryptStart","DecryptEnd","IsValid","Error"])

# Remove erroneous entries
attester = attester[attester['IsValid'] == 1].drop(columns = ["IsValid", "Error"])
attester

In [ ]:
# Summary of the gathered timespans
attester_spans = pd.DataFrame()

attester_spans["Size"] = attester["Size"] / 1024
attester_spans["MallocSpan"] = attester["MallocEnd"] - attester["MallocStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans["DecryptSpan"] = attester["DecryptEnd"] - attester["DecryptStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

attester_spans

In [ ]:
# Compute the mean and std
attester_malloc_export = compute_mean_std(attester_spans["Size"], attester_spans["MallocSpan"], 0.001)
attester_malloc_export

In [ ]:
# Compute the mean and std
attester_decrypt_export = compute_mean_std(attester_spans["Size"], attester_spans["DecryptSpan"], 0.001)
attester_decrypt_export

In [ ]:
#
## Verifier data
#

In [ ]:
# Import the datasets
verifier_columns = ["Size","Malloc1Start","Malloc1End","EncryptStart","EncryptEnd","Malloc2Start","Malloc2End"]
verifier = pd.read_csv("../logs/message3-time/verifier.csv", names = verifier_columns)

attester_for_errors = pd.read_csv("../logs/message3-time/attester.csv", \
        names = ["Size2","MallocStart2","MallocEnd2","DecryptStart","DecryptEnd","IsValid","Error"])

# Merge the two to identify which entries are erroneous in the verifier dataset
verifier = pd.concat([verifier, attester_for_errors], axis = 1)
verifier

In [ ]:
# Remove erroneous entries
verifier = verifier[verifier['IsValid'] == 1]

# Filter out attester columns
verifier = verifier[verifier_columns]
verifier

In [ ]:
verifier_spans = pd.DataFrame()

verifier_spans["Size"] = verifier["Size"] / 1024
verifier_spans["MallocSpan"] = verifier["Malloc1End"] - verifier["Malloc1Start"] \
                        + verifier["Malloc2End"] - verifier["Malloc2Start"] \
                        - 4*TIME_RETRIEVAL_TEE/2

verifier_spans["EncryptSpan"] = verifier["EncryptEnd"] - verifier["EncryptStart"] \
                        - 2*TIME_RETRIEVAL_TEE/2

verifier_spans

In [ ]:
# Compute the mean and std
verifier_malloc_export = compute_mean_std(verifier_spans["Size"], verifier_spans["MallocSpan"], 0.001)
verifier_malloc_export

In [ ]:
verifier_encrypt_export = compute_mean_std(verifier_spans["Size"], verifier_spans["EncryptSpan"], 0.001)
verifier_encrypt_export

In [ ]:
# Draw the area plot
area_plot_data = pd.DataFrame()
area_plot_data["AttesterMalloc"] = attester_malloc_export["Mean"]
area_plot_data["AttesterDecrypt"] = attester_decrypt_export["Mean"]
area_plot_data["VerifierEncrypt"] = verifier_encrypt_export["Mean"]
area_plot_data["VerifierMalloc"] = verifier_malloc_export["Mean"]

area_plot_data.plot.bar(stacked=True) #.area()

In [ ]:
# Export CSV
#
# File format: size, mean, stddev

attester_malloc_export.to_csv('../logs/message3-time/attester_malloc_formatted.csv')
attester_decrypt_export.to_csv('../logs/message3-time/attester_decrypt_formatted.csv')
verifier_malloc_export.to_csv('../logs/message3-time/verifier_malloc_formatted.csv')
verifier_encrypt_export.to_csv('../logs/message3-time/verifier_encrypt_formatted.csv')

In [ ]:
# Compute values for the paper

message3_minimum_time = verifier_encrypt_export.loc[verifier_encrypt_export.index == 0.5]["Mean"].values[0] \
                        + attester_decrypt_export.loc[attester_decrypt_export.index == 0.5]["Mean"].values[0]

message3_time_at_one_mb = verifier_encrypt_export.loc[verifier_encrypt_export.index == 1.0]["Mean"].values[0] \
                        + attester_decrypt_export.loc[attester_decrypt_export.index == 1.0]["Mean"].values[0]

message3_maximum_time = verifier_encrypt_export.loc[verifier_encrypt_export.index == 3.0]["Mean"].values[0] \
                        + attester_decrypt_export.loc[attester_decrypt_export.index == 3.0]["Mean"].values[0]

In [ ]:
# Export LaTeX

def write(file, name, number):
    f.write(f"\\def\\bmMessageThree{name}{{{number}}}\n")
        
f = open("../logs/message3-time/export.tex", "w")
write(f, "MinimumTime", f"{round(message3_minimum_time):,}")
write(f, "MaximumTime", f"{round(message3_maximum_time):,}")
write(f, "TimeAtOneMB", f"{formatForLatex(message3_time_at_one_mb, 1)}")
f.close()